I just wanted to practice implementing GANs, I don't know if GANs are superior or if GANs are implemented correctly.

Please don't feel bad.

If there are any oddities in this notebook, I would appreciate your pointing them out.

In [1]:
# For Google Colab
"""
from google.colab import drive
drive.mount('/content/drive')

# Install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

# Lib
from google.colab import files

# Please Upload `kaggle.json` file
uploaded = files.upload()

# Then copy kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

!kaggle competitions download -c tabular-playground-series-jun-2022
!unzip -o tabular-playground-series-jun-2022.zip -d tabular-playground-series-jun-2022
# !kaggle kernels output ehekatlact/tps2206-pytorch-lightning-fine-tuning2 -p ./DataSet
"""

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\n# Install kaggle packages\n!pip install -q kaggle\n!pip install -q kaggle-cli\n\n# Lib\nfrom google.colab import files\n\n# Please Upload `kaggle.json` file\nuploaded = files.upload()\n\n# Then copy kaggle.json into the folder where the API expects to find it.\n!mkdir -p ~/.kaggle\n!cp kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json\n!ls ~/.kaggle\n\n!kaggle competitions download -c tabular-playground-series-jun-2022\n!unzip -o tabular-playground-series-jun-2022.zip -d tabular-playground-series-jun-2022\n# !kaggle kernels output ehekatlact/tps2206-pytorch-lightning-fine-tuning2 -p ./DataSet\n"

In [2]:
%%capture
!pip install wandb
!pip install pytorch_lightning

In [3]:
import wandb
try:
    # add-ons -> secrets -> set your wandb api key
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Lib

In [4]:
# common
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import time, gc, string, math
from tqdm.notebook import tqdm
import warnings
import shutil
from collections import defaultdict
import heapq
import datetime
import random
from collections import OrderedDict
import glob
import copy

# sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

# pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
from torch.optim import lr_scheduler

# pytorch lightning
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger


In [5]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [6]:
# for google colab
# os.chdir("/content/drive/MyDrive/colab_data/TPS2206")

In [7]:
"""
os.makedirs('model', exist_ok=True)
shutil.rmtree('./model/')
os.makedirs('model', exist_ok=True)
"""

"\nos.makedirs('model', exist_ok=True)\nshutil.rmtree('./model/')\nos.makedirs('model', exist_ok=True)\n"

### Read DF

In [8]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col='row_id')
sub = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')

In [9]:
for col in data.columns:
    if "F_4" not in col:
        data[col] = data[col].fillna(data[col].mean())

In [10]:
na_col_list = []
for col in data.columns:
    if data[col].isna().sum() != 0:
        na_col_list.append(col)

In [11]:
na_index_of = {}
for col in na_col_list:
    na_index = list(np.where(data[col].isnull())[0])
    na_index_of[col] = na_index

In [12]:
cat_col_list = []
val_col_list = []
le = LabelEncoder()
for col in data.columns:
    if data[col].nunique() < 32:
        cat_col_list.append(col)
        data[col] = le.fit_transform(data[col])
    else:
        val_col_list.append(col)

In [13]:
data.fillna(data.mean(), inplace=True)

## Pytorch

### DataSet and DataLoader

In [14]:
class TrainDataset(Dataset):
    def __init__(self, X_val, X_cat, X_na, na_col_list=None):
        self.X_val = X_val
        self.X_cat = X_cat
        self.X_na = X_na
        self.na_col_list = na_col_list

    def __len__(self):
        return len(self.X_na)

    def __getitem__(self, item):
        # pre_val_inputs = copy.deepcopy(self.X_val[item])
        pre_val_inputs = self.X_val[item].copy()
        i = 0
        if self.na_col_list is not None:
            # val_inputsのうちいずれか1つを乱数で埋める
            # x = -1
            x = np.random.randn()
            i = random.randrange(0, len(na_col_list))
            pre_val_inputs[self.na_col_list[i]] = x
        val_inputs = torch.tensor(pre_val_inputs, dtype=torch.float32)
        cat_inputs = torch.tensor(self.X_cat[item], dtype=torch.int32)
        outputs = torch.tensor(self.X_na[item], dtype=torch.float32)
        zeros = torch.zeros(1, dtype=torch.float32)
        ones = torch.ones(1, dtype=torch.float32)

        return val_inputs, cat_inputs, outputs, zeros, ones, i

In [15]:
data_val = data.loc[:, val_col_list].values
data_cat = data.loc[:, cat_col_list].values
data_na = data.loc[:, na_col_list].values
ds = TrainDataset(data_val, data_cat, data_na, na_col_list)
for v, c, o, zero, ones, i in ds:
    break
del data_val, data_cat, data_na
gc.collect()

21

In [16]:
class DataModule(pl.LightningDataModule):
    # train, val, testの3つのDataLoaderを定義する
    # trainerにこれを渡すと、train, val, testのそれぞれのステップでこれを渡してくれる
    def __init__(self, train, valid, cat_col_list, val_col_list, na_col_list, batch_size):
        self.train_val = train.loc[:, val_col_list].values
        self.valid_val = valid.loc[:, val_col_list].values
        self.train_cat = train.loc[:, cat_col_list].values
        self.valid_cat = valid.loc[:, cat_col_list].values
        self.train_na = train.loc[:, na_col_list].values
        self.valid_na = valid.loc[:, na_col_list].values
        self.batch_size = batch_size
        self.na_col_list = [val_col_list.index(col) for col in na_col_list]
        self._log_hyperparams = None  # ナニコレ・・・

    def train_dataloader(self):
        ds = TrainDataset(self.train_val, self.train_cat, self.train_na, self.na_col_list)
        dl = DataLoader(ds, batch_size=self.batch_size, shuffle=True, pin_memory=True, drop_last=True, num_workers=CFG.num_workers, persistent_workers=True)
        return dl

    def val_dataloader(self):
        ds = TrainDataset(self.valid_val, self.valid_cat, self.valid_na)
        dl = DataLoader(ds, batch_size=self.batch_size, shuffle=False, pin_memory=True, drop_last=False, num_workers=CFG.num_workers, persistent_workers=True)
        return dl

    def predict_dataloader(self):
        ds = TrainDataset(self.valid_val, self.valid_cat, self.valid_na)
        dl = DataLoader(ds, batch_size=self.batch_size, shuffle=False, pin_memory=True, drop_last=False, num_workers=CFG.num_workers, persistent_workers=True)
        return dl

    def prepare_data_per_node(self):
        # TODO 本来要らないはずなんだけど・・・
        pass

    def teardown(self, stage=None):
        torch.cuda.empty_cache()  # TODO: これであってるのか不明　何も出てこないんだよね
        gc.collect()

### Pytorch Model

In [17]:
class Generator(nn.Module):
    def __init__(self, val_input_size, cat_input_size, output_size):
        super().__init__()
        hidden_size = 100
        emb_dim = 8
        self.cat_input_size = cat_input_size
        _emb_list = [nn.Embedding(32, emb_dim) for _ in range(cat_input_size)]
        self.emb_list = nn.ModuleList(_emb_list)
        self.fc1 = nn.Linear(val_input_size+emb_dim*cat_input_size, hidden_size*4)
        self.bn1 = nn.BatchNorm1d(hidden_size*4)
        self.fc2 = nn.Linear(hidden_size*4, hidden_size*4)
        self.fc3 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc4 = nn.Linear(hidden_size*2, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)
    
    def forward(self, val_x, cat_x):
        # dropoutとbnの併用禁止
        # bnは活性化関数の前に
        embbed_list = []
        for i in range(self.cat_input_size):
            emb = self.emb_list[i]
            embbed_list.append(emb(cat_x[:, i]))
        embbed = torch.cat(embbed_list, dim=1)
        x = torch.cat([val_x, embbed], dim=1)
        x = F.silu(self.bn1((self.fc1(x))))
        x = F.silu(self.fc2(x))
        x = F.silu(self.fc3(x))
        x = F.silu(self.fc4(x))
        x = self.fc5(x)
        return x

In [18]:
class Discriminator(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        hidden_size = 50
        self.fc1 = nn.Linear(input_size, hidden_size*8)
        self.bn1 = nn.BatchNorm1d(hidden_size*8)
        self.fc2 = nn.Linear(hidden_size*8, hidden_size*4)
        self.fc3 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc4 = nn.Linear(hidden_size*2, hidden_size)
        self.fc5 = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # dropoutとbnの併用禁止
        # bnは活性化関数の前に
        x = F.silu(self.bn1((self.fc1(x))))
        x = F.silu(self.fc2(x))
        x = F.silu(self.fc3(x))
        x = F.silu(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))
        return x

In [19]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat,y))

In [20]:
class NNModel(pl.LightningModule):
    # https://pytorch-lightning.readthedocs.io/en/stable/notebooks/lightning_examples/basic-gan.html
    def __init__(self, generator: nn.Module, discriminator: nn.Module):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.criterion_gen = RMSELoss()
        self.criterion_dis = nn.BCELoss()
        self.lr = CFG.lr

    def forward(self, val_x, cat_x) -> torch.Tensor:
        gen = self.generator(val_x, cat_x)
        return gen

    # Setup Optimizer and Scheduler
    def configure_optimizers(self):
        gen_model_params = [p for n, p in self.generator.named_parameters()]
        dis_model_params = [p for n, p in self.discriminator.named_parameters()]
        gen_optimizer_params = [
            {"params":  gen_model_params,
             "weight_decay": CFG.weight_decay,
             "lr": 1e-3
            },
        ]
        dis_optimizer_params = [
            {"params":  dis_model_params,
             "weight_decay": CFG.weight_decay,
             "lr": 1e-2
            },
        ]

        gen_optimizer = optim.Adam(gen_optimizer_params)
        dis_optimizer = optim.Adam(dis_optimizer_params)

        gen_scheduler = lr_scheduler.CosineAnnealingLR(dis_optimizer,
                                                T_max=2000,
                                                eta_min=1e-5,
                                                )
        interval = "step"

        dis_scheduler = lr_scheduler.CosineAnnealingLR(gen_optimizer,
                                                T_max=3000,
                                                eta_min=1e-5,
                                                )
        interval = "step"

        return [gen_optimizer, dis_optimizer], [{"scheduler": gen_scheduler, "interval": interval}, {"scheduler": dis_scheduler, "interval": interval}]

    # training valid test steps
    def training_step(self, batch_data, batch_idx, optimizer_idx):
        # batch_data: DataModuleで定義したtrain_dataloaderの結果
        # 戻値: lossであることが必須(裏でoptimizerに渡すため)
        X_val, X_cat, X_na, zeros, ones, i = batch_data
        # train generator
        if optimizer_idx == 0:
            # 生成したデータが本物=zeroと判定されるほど良い
            gen = self(X_val, X_cat)
            dis = self.discriminator(gen)
            g_loss = self.criterion_dis(dis, zeros)
            rmse = self.criterion_gen(X_na[:, i], gen[:, i])
            tqdm_dict = {"g_loss": g_loss, "rmse": rmse}
            output = OrderedDict({"loss": g_loss+rmse, "progress_bar": tqdm_dict, "log": tqdm_dict})
        
        # train discriminator 
        else:  # optimizer_idx == 1:
            # 生成したデータが偽物=onesと判定されるほど良い
            # 本物のデータが本物=zerosと判定されるほど良い
            gen = self(X_val, X_cat)
            fake_dis = self.discriminator(gen)
            fake_loss = self.criterion_dis(fake_dis, ones)
            real_dis = self.discriminator(X_na)
            real_loss = self.criterion_dis(real_dis, zeros)
            loss = fake_loss+real_loss
            tqdm_dict = {"fake_loss": fake_loss, "real_loss": real_loss}
            output = OrderedDict({"loss": loss, "progress_bar": tqdm_dict, "log": tqdm_dict})

        return output

    def training_epoch_end(self, outputs):
        # 1epoch分の処理(全バッチの処理)のreturn値をlistで受け取る
        rmse_list = [x[0]['log']['rmse'] for x in outputs]   # genとdicの2つが返される
        g_loss_list = [x[0]['log']['g_loss'] for x in outputs]
        fake_loss_list = [x[1]['log']['fake_loss'] for x in outputs]
        real_loss_list = [x[1]['log']['real_loss'] for x in outputs]
        rmse = torch.stack(rmse_list).mean()
        g_loss = torch.stack(g_loss_list).mean()
        fake_loss = torch.stack(fake_loss_list).mean()
        real_loss = torch.stack(real_loss_list).mean()
        self.log('rmse', rmse, prog_bar=True)
        self.log('g_loss', g_loss, prog_bar=True)
        self.log('fake_loss', fake_loss, prog_bar=True)
        self.log('real_loss', real_loss, prog_bar=True)
        if (self.current_epoch+1) % CFG.print_epoch_freq == 0:
            print("epoch:", self.current_epoch, "rmse:", rmse.item(), "g_loss", g_loss.item(), "fake_loss", fake_loss.item(), "real_loss", real_loss.item())

    """
    def validation_step(self, batch_data, batch_idx):
        # 戻値: 任意の辞書
        pass

    def validation_epoch_end(self, outputs):
        pass
    """

    def predict_step(self, batch_data, batch_idx):
        # 実際に予測させるときに使う
        X_val, X_cat, X_na, _, _, _ = batch_data
        outputs = self(X_val, X_cat)
        outputs = outputs.squeeze()
        # criterionがwithLogit系の場合は、sigmoidを追加する。
        # outputs = torch.sigmoid(outputs)
        return outputs

In [21]:
class CFG:
    num_workers = 2  # colabは4, kaggleは2
    weight_decay=0
    scheduler_type="ReduceLR"
    print_epoch_freq=1
    max_epochs=5
    batch_size=1000
    lr = 1e-3
    min_lr = 1e-6
    loop_end = 20
    debug = False

if CFG.debug:
    CFG.max_epochs=1

In [22]:
checkpoint_path_of = defaultdict(str)

In [23]:
model_name_prefix = datetime.datetime.now().strftime('%m%d%H%M%S')

for loop in range(CFG.loop_end):
    train = data
    valid = data
    result_data = copy.deepcopy(data)

    dm = DataModule(train, valid, cat_col_list, val_col_list, na_col_list, CFG.batch_size)

    # create model
    pre_model_name = "model"+model_name_prefix + "_" +str(loop-1)
    model_name = "model"+model_name_prefix + "_" +str(loop)
    dirpath = "./model/"
    gen = Generator(len(val_col_list), len(cat_col_list), len(na_col_list))
    dis = Discriminator(len(na_col_list))
    model = NNModel(gen, dis)
    if checkpoint_path_of[pre_model_name] != "":
        print('read model')
        checkpint_path = checkpoint_path_of[pre_model_name]
        model.load_from_checkpoint(checkpoint_path, generator=gen, discriminator=dis)
        checkpoint = torch.load(checkpoint_path)

    # train
    logger = WandbLogger()
    logger.log_hyperparams(CFG.__dict__)
    callbacks = [
                # pl.callbacks.EarlyStopping('valid_avg_loss', patience=5),  # validation_epoch_endの戻値が10ターン改善がなかったら打ち止め
                pl.callbacks.ModelCheckpoint(dirpath="./model/", filename=model_name, save_last=True, save_weights_only=False),  # model保存の設定
                pl.callbacks.LearningRateMonitor(),  # ログに学習率を吐き出す設定
    ]
    trainer = pl.Trainer(accelerator="auto", devices="auto", max_epochs=CFG.max_epochs, logger=logger, callbacks=callbacks, enable_progress_bar=True)
    trainer.fit(model, datamodule=dm)
    wandb.finish()

    # load_best_model
    checkpoint_path = glob.glob(dirpath+model_name+"*.ckpt")[0]
    model.load_from_checkpoint(checkpoint_path, generator=gen, discriminator=dis)
    checkpoint = torch.load(checkpoint_path)
    checkpoint_path_of[model_name] = dirpath+model_name+".ckpt"

    # predict
    dm = DataModule(train, valid, cat_col_list, val_col_list, na_col_list, CFG.batch_size)
    results = trainer.predict(model=model, datamodule=dm)
    preds = []
    for batch in results:
        preds.append(batch)
    outputs = torch.cat(preds, dim=0)

    # result_data.loc[valid_index, na_col_list] = outputs.tolist()
    result_data.loc[:, na_col_list] = outputs.tolist()

    # fill_na
    for col in na_col_list:
        na_index = na_index_of[col]
        data.loc[na_index, col] = result_data.loc[na_index, col]

    gc.collect()


wandb: Currently logged in as: ehekatlact. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")


Training: 0it [00:00, ?it/s]

epoch: 0 rmse: 1.8456164598464966 g_loss 2.3726792335510254 fake_loss 0.38531509041786194 real_loss 0.42660748958587646
epoch: 1 rmse: 0.8682112097740173 g_loss 0.8784027695655823 fake_loss 0.621319055557251 real_loss 0.6700441837310791
epoch: 2 rmse: 0.711259663105011 g_loss 1.4941807985305786 fake_loss 0.4906647801399231 real_loss 0.543310821056366
epoch: 3 rmse: 0.7295204997062683 g_loss 2.331308603286743 fake_loss 0.31281837821006775 real_loss 0.3334687650203705
epoch: 4 rmse: 0.6873146891593933 g_loss 1.5322061777114868 fake_loss 0.44219648838043213 real_loss 0.46689313650131226


epoch,▁▃▅▆█
fake_loss,▃█▅▁▄
g_loss,█▁▄█▄
lr-Adam,███▇▇▆▆▅▄▄▃▂▂▁▁▁▁▁▁▂▂▃▃▄▅▆▆▇▇█████▇▇▇▆▅▅
lr-Adam-1,████▇▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▂▂▂▃▃▃▄▄▅▅▆▆
real_loss,▃█▅▁▄
rmse,█▂▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
epoch,5
fake_loss,0.4422
g_loss,1.53221


...

Training: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f89bc8445f0>
Traceback (most recent call last):
Exception ignored in:   File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    <function _MultiProcessingDataLoaderIter.__del__ at 0x7f89bc8445f0>self._shutdown_workers()

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
      File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
if w.is_alive():    if w.is_alive():

  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only 

epoch: 0 rmse: 0.26864007115364075 g_loss 0.6931472420692444 fake_loss 0.6931472420692444 real_loss 0.6931472420692444
epoch: 1 rmse: 0.2570686340332031 g_loss 0.6931472420692444 fake_loss 0.6931472420692444 real_loss 0.6931472420692444
epoch: 2 rmse: 0.2456239014863968 g_loss 0.6931472420692444 fake_loss 0.6931472420692444 real_loss 0.6931472420692444
epoch: 3 rmse: 0.24487482011318207 g_loss 0.6931472420692444 fake_loss 0.6931472420692444 real_loss 0.6931472420692444
epoch: 4 rmse: 0.25491952896118164 g_loss 0.6931472420692444 fake_loss 0.6931472420692444 real_loss 0.6931472420692444


epoch,▁▃▅▆█
fake_loss,▁▁▁▁▁
g_loss,▁▁▁▁▁
lr-Adam,███▇▇▆▆▅▄▄▃▂▂▁▁▁▁▁▁▂▂▃▃▄▅▆▆▇▇█████▇▇▇▆▅▅
lr-Adam-1,████▇▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▂▂▂▃▃▃▄▄▅▅▆▆
real_loss,▁▁▁▁▁
rmse,█▅▁▁▄
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
epoch,5
fake_loss,0.69315
g_loss,0.69315


Predicting: 1000it [00:00, ?it/s]

In [24]:
result_data.describe()

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_2_0,F_2_1,F_2_2,F_2_3,F_2_4,F_2_5,F_2_6,F_2_7,F_2_8,F_2_9,F_2_10,F_2_11,F_2_12,F_2_13,F_2_14,F_2_15,F_2_16,F_2_17,F_2_18,F_2_19,F_2_20,F_2_21,F_2_22,F_2_23,F_2_24,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,F_3_10,F_3_11,F_3_12,F_3_13,F_3_14,F_3_15,F_3_16,F_3_17,F_3_18,F_3_19,F_3_20,F_3_21,F_3_22,F_3_23,F_3_24,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,-0.000687,0.002091,0.000551,0.000982,0.002437,0.000635,-0.000124,-0.063857,-0.000014,0.000451,0.000185,-0.001129,-0.061151,-0.067064,-0.000905,2.687703,2.514404,0.976757,2.517147,2.941194,1.532953,1.491746,2.645875,1.177792,1.111291,3.279969,2.465888,2.759102,2.479939,1.717679,1.779894,1.801325,1.243386,1.557055,1.603460,2.230707,2.032374,1.605533,0.708911,3.133517,0.001737,-0.001147,0.000605,0.000834,0.001290,-0.002180,0.000058,0.001525,0.000773,-0.000440,0.001706,0.000733,0.000259,-0.002459,0.000727,-0.001513,-0.000665,-0.000214,0.000063,-0.064909,0.002370,-0.059327,0.000087,0.000365,-0.000817,0.290455,-0.323649,-0.137975,-0.210620,0.255882,0.432449,0.051606,0.334231,-0.089615,-0.075955,0.024391,0.672646,0.245806,0.282364,0.043161
std,0.991536,0.990729,0.990782,0.990682,0.991937,0.990842,0.991315,0.719375,0.991586,0.991025,0.990423,0.990567,0.705536,0.739532,0.990752,1.875394,1.751238,1.038280,1.653054,1.976869,1.351220,1.315535,1.744770,1.318677,1.095853,1.872243,1.600069,1.704955,1.646004,1.557458,1.458558,1.458531,1.246472,1.444577,1.420668,1.561235,1.609266,1.555289,1.079393,1.820703,0.992256,0.991248,0.989879,0.991293,0.990786,0.990928,0.990213,0.991644,0.991904,0.992537,0.990811,0.990233,0.991037,0.991185,0.989830,0.991333,0.990828,0.990392,0.991015,0.732668,0.989670,0.690410,0.990050,0.990770,0.990989,2.320868,2.398355,0.836836,0.823857,2.377187,2.353634,2.278469,2.358081,0.773724,0.780682,0.703400,5.006194,2.379984,2.369494,0.777808
min,-4.664447,-4.790594,-4.870645,-5.053023,-5.363454,-5.508223,-5.199170,-6.990302,-4.566593,-4.997549,-4.794973,-4.612384,-7.063291,-6.895633,-4.630454,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.693600,-4.466075,-4.885859,-4.677193,-5.009163,-4.870645,-5.018589,-5.053023,-5.508223,-4.846381,-4.626277,-4.597624,-4.533044,-4.746642,-5.363454,-4.448034,-4.821627,-4.806190,-5.199170,-6.069162,-4.997549,-7.146564,-4.740668,-5.251064,-4.891499,-12.840856,-12.743421,-10.151258,-8.748049,-13.320487,-12.494095,-10.684401,-11.705502,-9.876698,-8.506112,-10.719352,-26.298292,-11.442402,-10.857075,-8.996425
25%,-0.6597

In [25]:
result_data

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_2_0,F_2_1,F_2_2,F_2_3,F_2_4,F_2_5,F_2_6,F_2_7,F_2_8,F_2_9,F_2_10,F_2_11,F_2_12,F_2_13,F_2_14,F_2_15,F_2_16,F_2_17,F_2_18,F_2_19,F_2_20,F_2_21,F_2_22,F_2_23,F_2_24,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,F_3_10,F_3_11,F_3_12,F_3_13,F_3_14,F_3_15,F_3_16,F_3_17,F_3_18,F_3_19,F_3_20,F_3_21,F_3_22,F_3_23,F_3_24,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,-1.416075,1.201521,0.551902,-0.759827,-0.000905,2,2,0,3,2,1,1,3,2,2,3,3,2,5,4,1,0,1,0,2,1,2,0,1,2,-0.240522,0.061529,0.561090,-0.171943,-0.431996,0.473508,0.596924,0.819306,1.479061,1.264616,-1.116881,0.759443,-0.086915,-0.620685,0.057216,1.076380,-0.780608,-1.940907,-0.717021,0.599093,0.498347,0.118770,-0.228913,0.000365,0.301610,5.488440,1.037650,-0.208374,-0.090677,-0.826030,3.840709,0.894307,0.267267,-0.598939,0.154083,0.391101,6.294586,3.773519,1.106589,1.205772
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,-0.853630,0.674525,0.843058,-0.041438,0.259496,3,5,2,1,4,2,3,2,0,0,9,3,1,4,1,2,1,0,1,1,1,1,3,2,5,-0.446068,1.433358,0.040099,-1.994062,-0.602324,-0.611391,-1.151884,0.065485,0.352023,-0.843751,1.167272,0.921445,-0.839827,0.759015,-1.547387,0.720435,-0.944045,1.796462,-1.046357,-0.581515,0.704543,0.375222,0.705963,0.032771,-0.000817,-1.775020,-1.112791,-0.630600,-1.517282,1.221322,-2.873559,-0.687622,2.351553,-0.116715,0.805895,-0.346982,5.413508,-4.370509,-2.794813,-0.072805
2,0.256023,-1.059874,0.000551,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,-0.073235,-1.381605,-0.108676,0.703693,-0.464042,3,2,1,3,2,1,0,6,1,1,3,4,1,2,2,1,2,2,4,8,1,5,1,2,4,-0.770592,0.483139,-0.636484,-1.305018,-2.089889,0.276761,-1.208640,-0.855769,0.232363,0.215841,-1.031405,0.582437,-0.314639,-0.497409,0.489356,0.915049,-0.513410,0.904206,-0.056089,0.212927,-0.574126,-1.517749,-0.888472,0.142264,1.000822,1.806508,3.888281,0.402665,0.335418,0.359227,2.328988,-0.082672,2.443724,0.316152,-0.254604,-0.034994,-0.132358,-2.251357,3.640579,-0.183697
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,-0.618973,0.742112,0.494157,0.744673,-0.769181,2,3,2,1,1,1,2,0,6,2,6,2,1,1,2,3,1,2,1,2,6,2,1,0,2,-0.691020,-1.003026,0.868989,0.464920,0.001290,0.499838,0.203723,-0.451576,-1.233499,0.903493,1.144558,-1.479893,-0.414316,0.024820,-0.976814,0.176633,-0.940022,-1.918049,1.506448,0.604388,0.578250,-0.122676,-0.711811,0.228523,0.999599,-2.735969,0.438517,0.754595,-0.954724,2.553473,2.108816,-4.502384,-0.764086,-0.305508,0.664243,-0.648340,8.714813,-4.936552,-3.089380,0.488261
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,1.338692,1.193330,1.038010,-0.763692,0.513466,5,0,1,3,3,0,2,5,1,3,5,5,4,4,2,0,2,3,2,5,2,2,1,0,4,1.196402,1.076537,0.486482,0.424305,0.475876,0.426978,-1.668823,-0.843056,1.008980,-0.704152,-0.894834,1.491608,-0.149938,1.256330,2.427308,0.878677,0.594314,0.380175,0.666499,-0.664403,1.276316,-0.059327,-1.276574,-0.768874,1.852815,-0.160582,-0.464384,1.097845,-0.767652,-0.503668,1.122928,2.580275,3.386840,0.820562,0.641883,0.567147,-5.860733,-2.908807,3.670837,-0.767023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.823740,0.285673,0.343307,-0.436747,1.700549,-1.069432,0.819698,-0.168457,-0.429074,0.844075,0.569298,0.235662,0.540328,-0.262106,-1.566722,4,8,1,4,4,0,1,1,2,1,2,1,0,4,1,1,1,1,2,2,3,0,3,0,6,0.

In [26]:
data

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_2_0,F_2_1,F_2_2,F_2_3,F_2_4,F_2_5,F_2_6,F_2_7,F_2_8,F_2_9,F_2_10,F_2_11,F_2_12,F_2_13,F_2_14,F_2_15,F_2_16,F_2_17,F_2_18,F_2_19,F_2_20,F_2_21,F_2_22,F_2_23,F_2_24,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,F_3_10,F_3_11,F_3_12,F_3_13,F_3_14,F_3_15,F_3_16,F_3_17,F_3_18,F_3_19,F_3_20,F_3_21,F_3_22,F_3_23,F_3_24,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,-1.416075,1.201521,0.551902,-0.759827,-0.000905,2,2,0,3,2,1,1,3,2,2,3,3,2,5,4,1,0,1,0,2,1,2,0,1,2,-0.240522,0.061529,0.561090,-0.171943,-0.431996,0.473508,0.596924,0.819306,1.479061,1.264616,-1.116881,0.759443,-0.086915,-0.620685,0.057216,1.076380,-0.780608,-1.940907,-0.717021,0.599093,0.498347,0.118770,-0.228913,0.000365,0.301610,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,-0.853630,0.674525,0.843058,-0.041438,0.259496,3,5,2,1,4,2,3,2,0,0,9,3,1,4,1,2,1,0,1,1,1,1,3,2,5,-0.446068,1.433358,0.040099,-1.994062,-0.602324,-0.611391,-1.151884,0.065485,0.352023,-0.843751,1.167272,0.921445,-0.839827,0.759015,-1.547387,0.720435,-0.944045,1.796462,-1.046357,-0.581515,0.704543,0.375222,0.705963,0.032771,-0.000817,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,0.000551,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,-0.073235,-1.381605,-0.108676,0.703693,-0.464042,3,2,1,3,2,1,0,6,1,1,3,4,1,2,2,1,2,2,4,8,1,5,1,2,4,-0.770592,0.483139,-0.636484,-1.305018,-2.089889,0.276761,-1.208640,-0.855769,0.232363,0.215841,-1.031405,0.582437,-0.314639,-0.497409,0.489356,0.915049,-0.513410,0.904206,-0.056089,0.212927,-0.574126,-1.517749,-0.888472,0.142264,1.000822,1.914908,3.877128,0.402665,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,-0.618973,0.742112,0.494157,0.744673,-0.769181,2,3,2,1,1,1,2,0,6,2,6,2,1,1,2,3,1,2,1,2,6,2,1,0,2,-0.691020,-1.003026,0.868989,0.464920,0.001290,0.499838,0.203723,-0.451576,-1.233499,0.903493,1.144558,-1.479893,-0.414316,0.024820,-0.976814,0.176633,-0.940022,-1.918049,1.506448,0.604388,0.578250,-0.122676,-0.711811,0.228523,0.999599,-2.638262,0.546676,0.865400,-0.857077,2.667105,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,1.338692,1.193330,1.038010,-0.763692,0.513466,5,0,1,3,3,0,2,5,1,3,5,5,4,4,2,0,2,3,2,5,2,2,1,0,4,1.196402,1.076537,0.486482,0.424305,0.475876,0.426978,-1.668823,-0.843056,1.008980,-0.704152,-0.894834,1.491608,-0.149938,1.256330,2.427308,0.878677,0.594314,0.380175,0.666499,-0.664403,1.276316,-0.059327,-1.276574,-0.768874,1.852815,-0.230342,-0.459019,1.128705,-0.748683,-0.503668,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.823740,0.285673,0.343307,-0.436747,1.700549,-1.069432,0.819698,-0.168457,-0.429074,0.844075,0.569298,0.235662,0.540328,-0.262106,-1.566722,4,8,1,4,4,0,1,1,2,1,2,1,0,4,1,1,1,1,2,2,3,0,3,0,6,0.0

In [27]:
ind_list = []
val_list = []
for i in tqdm(sub.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    val = data[col][row]
    ind_list.append(i)
    val_list.append(val)

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [28]:
sub['value'].loc[ind_list] = val_list

In [29]:
sub.to_csv("submission.csv", index=True)
sub

,value
row-col,
0-F_1_14,-0.000905
0-F_3_23,0.000365
1-F_3_24,-0.000817
2-F_1_2,0.000551
2-F_4_2,0.402665
...,...
999993-F_4_2,-0.562760
999994-F_3_10,0.001706
999994-F_4_9,-0.203973
